<a href="https://colab.research.google.com/github/greyhound101/coco/blob/main/coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html


In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/coco/annotations_trainval2017.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
sys.argv[1]

In [5]:
from pycocotools.coco import COCO
import cv2
import numpy as np
from os.path import join, isdir
from os import mkdir, makedirs
from concurrent import futures
import sys
import time
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import json
import sys


def filter_coco(coco, cls_split):
    new_anns = []
    all_cls_dict = {}
    for img_id, id in enumerate(coco.imgs):
        img = coco.loadImgs(id)[0]
        anns = coco.loadAnns(coco.getAnnIds(imgIds=id, iscrowd=None))
        skip_flag = False
        img_cls_dict = {}
        if len(anns) == 0:
            continue
        for ann in anns:
            segmentation = ann['segmentation']
            area = ann['area']
            iscrowd = ann['iscrowd']
            image_id = ann['image_id']
            bbox = ann['bbox']
            category_id = ann['category_id']
            id = ann['id']
            bbox_area = bbox[2] * bbox[3]
            
            # filter images with small boxes
            if category_id in cls_split:
                if bbox_area < 32 * 32:
                    skip_flag = True
                
        if skip_flag:
            continue
        else:
            for ann in anns:
                category_id = ann['category_id']
                if category_id in cls_split:
                    new_anns.append(ann)

                    if category_id in all_cls_dict.keys():
                        all_cls_dict[category_id] += 1
                    else:
                        all_cls_dict[category_id] = 1
                        
    print(len(new_anns))
    print(sorted(all_cls_dict.items(), key = lambda kv:(kv[1], kv[0])))     
    return new_anns


root_path = '/content/-f'
print(root_path)
#root_path = '/home/fanqi/data/COCO'
dataDir = '/content/annotations'
support_dict = {}

support_dict['support_box'] = []
support_dict['category_id'] = []
support_dict['image_id'] = []
support_dict['id'] = []
support_dict['file_path'] = []

voc_inds = (0, 1, 2, 3, 4, 5, 6, 8, 14, 15, 16, 17, 18, 19, 39, 56, 57, 58, 60, 62)


for dataType in ['instances_train2017.json']: #, 'split_voc_instances_train2017.json']:
    annFile = join(dataDir, dataType)

    with open(annFile,'r') as load_f:
        dataset = json.load(load_f)
        print(dataset.keys())
        save_info = dataset['info']
        save_licenses = dataset['licenses']
        save_images = dataset['images']
        save_categories = dataset['categories']
        save_annotations = dataset['annotations']


    inds_split2 = [i for i in range(len(save_categories)) if i not in voc_inds]

    # split annotations according to categories
    categories_split1 = [save_categories[i] for i in voc_inds]
    categories_split2 = [save_categories[i] for i in inds_split2]
    cids_split1 = [c['id'] for c in categories_split1]
    cids_split2 = [c['id'] for c in categories_split2]
    print('Split 1: {} classes'.format(len(categories_split1)))
    for c in categories_split1:
        print('\t', c['name'])
    print('Split 2: {} classes'.format(len(categories_split2)))
    for c in categories_split2:
        print('\t', c['name'])
    
    coco = COCO(annFile)

    # for non-voc, there can be non_voc images
    annotations_split2 = filter_coco(coco, cids_split2)

    # for voc, there can be non_voc images
    annotations = dataset['annotations']
    annotations_split1 = []
    
    for ann in annotations:
        if ann['category_id'] in cids_split1: # voc 20
            annotations_split1.append(ann)

    dataset_split1 = {
        'info': save_info,
        'licenses': save_licenses,
        'images': save_images,
        'annotations': annotations_split1,
        'categories': save_categories}
    dataset_split2 = {
        'info': save_info,
        'licenses': save_licenses,
        'images': save_images,
        'annotations': annotations_split2,
        'categories': save_categories}
    new_annotations_path = os.path.join(root_path, 'new_annotations')
    if not os.path.exists(new_annotations_path):
        os.makedirs(new_annotations_path)
    split2_file = os.path.join(root_path, 'new_annotations/final_split_non_voc_instances_train2017.json')

    with open(split2_file, 'w') as f:
        json.dump(dataset_split2, f) 

/content/-f
dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])
Split 1: 20 classes
	 person
	 bicycle
	 car
	 motorcycle
	 airplane
	 bus
	 train
	 boat
	 bird
	 cat
	 dog
	 horse
	 sheep
	 cow
	 bottle
	 chair
	 couch
	 potted plant
	 dining table
	 tv
Split 2: 60 classes
	 truck
	 traffic light
	 fire hydrant
	 stop sign
	 parking meter
	 bench
	 elephant
	 bear
	 zebra
	 giraffe
	 backpack
	 umbrella
	 handbag
	 tie
	 suitcase
	 frisbee
	 skis
	 snowboard
	 sports ball
	 kite
	 baseball bat
	 baseball glove
	 skateboard
	 surfboard
	 tennis racket
	 wine glass
	 cup
	 fork
	 knife
	 spoon
	 bowl
	 banana
	 apple
	 sandwich
	 orange
	 broccoli
	 carrot
	 hot dog
	 pizza
	 donut
	 cake
	 bed
	 toilet
	 laptop
	 mouse
	 remote
	 keyboard
	 cell phone
	 microwave
	 oven
	 toaster
	 sink
	 refrigerator
	 book
	 clock
	 vase
	 scissors
	 teddy bear
	 hair drier
	 toothbrush
loading annotations into memory...
Done (t=18.44s)
creating index...
index created!
148472
[(80,

In [45]:
    from PIL import Image
    import urllib 
    dataDir = '.'
    #root_path = '/home/fanqi/data/COCO'
    root_path = '/content/-f'
    support_path = os.path.join(root_path, 'support')
    if not isdir(support_path): 
        mkdir(support_path)
    #else:
    #    shutil.rmtree(support_path)

    support_dict = {}
    
    support_dict['support_box'] = []
    support_dict['category_id'] = []
    support_dict['image_id'] = []
    support_dict['id'] = []
    support_dict['file_path'] = []

    for dataType in ['train2017']: #, 'train2017']:
        set_crop_base_path = join(support_path, dataType)
        set_img_base_path = join(dataDir, dataType)

        annFile = os.path.join(root_path, 'new_annotations/final_split_non_voc_instances_train2017.json')
        with open(annFile,'r') as load_f:
            dataset = json.load(load_f)
            print(dataset.keys())
            save_info = dataset['info']
            save_licenses = dataset['licenses']
            save_images = dataset['images']
            save_categories = dataset['categories']

        coco = COCO(annFile)
        
        for img_id, id in enumerate(coco.imgs):
            if img_id % 100 == 0:
                print(img_id)
            img = coco.loadImgs(id)[0]
            anns = coco.loadAnns(coco.getAnnIds(imgIds=id, iscrowd=None))

            if len(anns) == 0:
                continue

            frame_crop_base_path = join(set_crop_base_path, img['file_name'].split('/')[-1].split('.')[0])
            if not isdir(frame_crop_base_path): makedirs(frame_crop_base_path)
            print('{}/{}'.format(set_img_base_path, img['file_name']))
            im = np.array(Image.open(urllib.request.urlopen(img['coco_url'])))
            #print('{}/{}'.format(set_img_base_path, img['file_name']))
            for item_id, ann in enumerate(anns):
                rect = ann['bbox']
                bbox = [rect[0], rect[1], rect[0] + rect[2], rect[1] + rect[3]]
                support_img, support_box = crop_support(im, bbox)
                #im_name = img['file_name'].split('.')[0] + '_' + str(item_id) + '.jpg'
                #output_dir = './fig'
                #vis_image(support_img[:, :, ::-1], support_box, join(frame_crop_base_path, '{:04d}.jpg'.format(item_id)))
                if rect[2] <= 0 or rect[3] <=0:
                    print(rect)
                    continue
                file_path = join(frame_crop_base_path, '{:04d}.jpg'.format(item_id))
                cv2.imwrite(file_path, support_img)
                #print(file_path)
                support_dict['support_box'].append(support_box.tolist())
                support_dict['category_id'].append(ann['category_id'])
                support_dict['image_id'].append(ann['image_id'])
                support_dict['id'].append(ann['id'])
                support_dict['file_path'].append(file_path)

        support_df = pd.DataFrame.from_dict(support_dict)

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])
loading annotations into memory...
Done (t=6.07s)
creating index...
index created!
0
./train2017/000000522418.jpg
./train2017/000000184613.jpg
./train2017/000000318219.jpg
./train2017/000000060623.jpg
./train2017/000000222564.jpg
./train2017/000000483108.jpg
./train2017/000000086408.jpg
./train2017/000000386164.jpg
./train2017/000000384553.jpg
./train2017/000000012448.jpg
./train2017/000000515289.jpg
./train2017/000000383445.jpg
./train2017/000000032992.jpg
./train2017/000000540763.jpg
100
./train2017/000000019358.jpg
./train2017/000000459912.jpg
./train2017/000000394240.jpg
./train2017/000000074331.jpg
./train2017/000000452684.jpg
./train2017/000000339974.jpg
./train2017/000000368117.jpg
./train2017/000000219514.jpg
./train2017/000000521634.jpg
./train2017/000000227125.jpg
./train2017/000000439969.jpg
./train2017/000000029913.jpg
./train2017/000000410437.jpg
./train2017/000000431573.jpg
./train2017/000000127474.jpg

ValueError: ignored